In [1]:
from glob import glob
import os
routes = glob('/data/ephemeral/home/datas/text/*')

In [2]:
import re
from tqdm import tqdm
def parse_wiki_file(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 문서별로 분리 (각 문서는 <doc>로 시작하고 </doc>로 끝남)
    doc_pattern = re.compile(r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>', re.DOTALL)
    matches = doc_pattern.findall(content)
    
    for match in matches:
        doc_id, url, title, text = match
        documents.append({
            "id": doc_id,
            "url": url,
            "title": title,
            "text": text.strip()
        })
    
    return documents
result = []
for route in routes:
    for r in tqdm(glob(route + '/*'), desc = 'collecting...'):
        result.extend(parse_wiki_file(r))


collecting...:   0%|          | 0/100 [00:00<?, ?it/s]

collecting...: 100%|██████████| 100/100 [00:01<00:00, 59.77it/s]


In [3]:
import pandas as pd

df = pd.DataFrame(result)
df['doc_len'] = df['text'].apply(lambda x: len(x))

In [4]:
df = df[(df['doc_len'] > 100) & (df['doc_len'] < 10000)]

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(doc):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(doc)
    return chunks

df['splitted_text'] = df['text'].apply(text_split)

In [10]:
df['splitted_text'].sample(3)

606958     [안드레아스 쾨프케, 안드레아스 쾨프케(: Andreas Köpke, 1962년 3...
821034     [우에다역 (나가노현)\n\n우에다역(: 上田駅, うえだえき)은 일본 나가노현 우에...
1295171    [성교, 성교(性交) 또는 성관계(性關係, : sexual intercourse)는...
Name: splitted_text, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from kiwipiepy import Kiwi
from keybert import KeyBERT
from transformers import pipeline
from ast import literal_eval
from tqdm import tqdm

kw_model = KeyBERT(model='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 텍스트에서 주요 키워드 추출 함수 (KeyBERT 사용)
def extract_keywords(doc, top_n=5, ngram = 2):
    kiwi = Kiwi()
    # 문장을 형태소 분석하여 명사, 형용사, 동사 등을 추출
    tokens = kiwi.tokenize(doc)
    words = [token.form for token in tokens if token.tag.startswith(('NN', 'VV', 'VA'))]  # 명사(NN), 동사(VV), 형용사(VA)만 추출

    # KeyBERT 모델을 사용하여 주요 키워드 추출
    keywords = kw_model.extract_keywords(' '.join(words), keyphrase_ngram_range=(1, ngram), stop_words=None, top_n=top_n)
    
    # 결과에서 키워드만 추출하여 리스트로 반환
    return [keyword[0] for keyword in keywords]

train = pd.read_csv('datas/test.csv')
train['dic'] = train['problems'].apply(lambda x: literal_eval(x))
train['question'] = train['dic'].apply(lambda x: x['question'])


tqdm.pandas()

train['keywords'] = train.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question']), axis=1)
train['keywords_1'] = train.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question'], ngram = 1), axis=1)
# 위 코드는 ngram 2개까지 아래코드는 ngram 1개만

train[['id', 'paragraph', 'problems', 'question_plus', 'keywords', 'keywords_1']].to_csv('datas/train+keyword.csv', index = False)
train = pd.read_csv('datas/train+keyword.csv')

docs = []
titles_set = set(df['title'].values)  
cnt = 0

for words in tqdm(train['keywords_1'].values):
    for word in words:
        if word in titles_set:
            docs.append(df[df['title'] == word].to_dict('records'))

# 위키 문서에서 title이 매칭되는 애들만 뽑기

flattened_docs = [item for sublist in docs for item in sublist]

df = pd.DataFrame(flattened_docs)

,id,paragraph,problems,question_plus,dic,question
0,generation-for-nlp-0,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 . 독서의 즐거움에는 ...,"{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...",NaN,"{'question': '윗글의 내용과 일치하지 않는 것은?', 'choices'...",윗글의 내용과 일치하지 않는 것은?
1,generation-for-nlp-1,사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 . 독서의 즐거움에는 ...,{'question': '윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은...,NaN,{'question': '윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은...,윗글을 읽고 ㉠에 대해 보인 반응으로 적절하지 않은 것은?
2,generation-for-nlp-2,(가 ) 중국에서 비롯된 유서( 類書)는 고금의 서적에서 자료를 수집하고 항목별로...,"{'question': '(가 )와 (나 )에 대한 설명으로 가장 적절한 것은?',...",NaN,"{'question': '(가 )와 (나 )에 대한 설명으로 가장 적절한 것은?',...",(가 )와 (나 )에 대한 설명으로 가장 적절한 것은?


In [12]:
corpus = []
for i ,item in tqdm(df.iterrows()):
    corpus += item['splitted_text']
corpus = list(filter(lambda x: len(x) > 10, corpus))


511081it [00:22, 22878.26it/s]


In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import gc
import torch
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.schema import Document
gc.collect()
torch.cuda.empty_cache()
model_name = "intfloat/multilingual-e5-large-instruct"
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={'normalize_embeddings': True}
)

docs = [Document(page_content=text, metadata={"source": f"doc_{i}"}) for i, text in enumerate(corpus)]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2

faiss_vectorstore = FAISS.from_documents(docs, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})
ensemble_retriever = EnsembleRetriever(
retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)   



In [182]:

train['dic'] = train['problems'].apply(lambda x: literal_eval(x))
train['question'] = train['dic'].apply(lambda x: x['question'])

In [183]:
idx = 4
result = ensemble_retriever.get_relevant_documents(train['paragraph'][idx] + train['question'][idx])
print(train['paragraph'][idx])
print(train['question'][idx])
print(result)

선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가)이/가 크게 노하여 한성부의 조례(皂隷)와 병졸로 하여 금 한 강 밖으로 몰아내게 하고 드디어 천여 곳의 서원을 철폐하고 그 토지를 몰수하여 관에 속하게 하였다.－대한계년사 －
(가) 인물이 추진한 정책으로 옳지 않은 것은?
[Document(metadata={'source': 'doc_25017'}, page_content='하고 그 위에 주거나 신전·궁전을 만든다. 이러한 일이 반복되어 수십 층으로 퇴적이 되는 것이다.'), Document(metadata={'source': 'doc_50729'}, page_content='영국해협에서 내륙으로 15km 들어간 지점의 오른 강 연안에 위치한다. 오른 강과 운하를 통해 영국 해협과 연결되는 항구도시이다. 11세기 정복왕 윌리엄(윌리엄 1세) 시대 때 노르망디의 한 중심지가 되었고, 그도 이 곳에 묻혔다. 1346년 에드워드 3세가 정복하여 잉글랜드의 지배하에 있다가 1450년 프랑스로 넘어왔다. 제2차 세계대전 중 노르망디 상륙 작전 때 이 도시는 중요한 요충지 중 한 곳이었으며, 크게 파괴되었다가 전쟁 후 근대적인 도시로 재건되었다. 11세기에 건립되어 윌리엄 왕과 왕비 마틸다가 각각 묻힌 생테티엔')]


In [184]:
from tqdm import tqdm

def generate_hint(row):
    query = row['question'] + ' ' + row['question']
    relevant_docs = ensemble_retriever.get_relevant_documents(query)
    
    # `hint` 문자열 생성
    hint = ''
    for i, relevant_doc in enumerate(relevant_docs):
        hint += f"참고문서{i} : {relevant_doc.page_content}\n"
        if i == 2:
            break  # 3개까지만
    return hint

# tqdm 적용 및 `apply`를 사용하여 새 열 생성
tqdm.pandas()
train['hint'] = train.progress_apply(generate_hint, axis=1)


100%|██████████| 2031/2031 [13:45<00:00,  2.46it/s]


In [186]:
d = pd.read_csv('datas/train+klue.csv')
d['hint'] = train['hint']

In [187]:
print(d['hint'][4])

참고문서0 : 이 때에 자기의 의로우심을 나타내사 자기도 의로우시며 또한 예수 믿는 자를 의롭다 하려 하심이니라 그런즉 자랑할 데가 어디뇨 있을 수가 없느니라 무슨 법으로냐 행위로냐 아니라 오직 믿음의 법으로니라 그러므로 사람이 의롭다 하심을 얻는 것은 율법의 행위에 있지 않고 믿음으로 되는줄 우리가 인정하노라”
참고문서1 : 이렇게 사찰의 건축물로서 지어졌던 탑도 있지만 민간에서 단순하게 주변에 있던 돌을 쌓아 올려 만든 돌탑도 있다. 소박하게 만들어진 이런 돌탑들은 토속신앙과 관련이 깊어 서낭신을 모시는 서낭당과 가까이 있는 경우가 많았으며, 기복의 용도로 이용되었다. 한국의 불탑은 중국과 일본에 비해 그 규모가 작은 편이다. 이는 오랜 전란으로 인한 문화재의 훼손이 잦았고, 조선의 숭유억불 정책으로 인해 의도적으로 사찰을 파괴하는 경우가 많았기 때문이다.



In [189]:
d[['id', 'paragraph', 'problems', 'question_plus', 'klue', 'hint']].to_csv('train+klue+hint.csv')